# Temporal Simulation

Assuming stationary processes, we used the previously demonstrated (check other examples)
average QoS metrics over time by simulating for a very large `max_time`. Here, our goal is to find
temporal characteristics by calculating sample averages (with limited time executation).
To do so, we need to provide the function instances in the current state.

In [1]:
from pacssim.SimProcess import ExpSimProcess
from pacssim.FunctionInstance import FunctionInstance
from pacssim.ServerlessTemporalSimulator import ServerlessTemporalSimulator

from tqdm import tqdm

cold_service_rate = 1/2.163
warm_service_rate = 1/2.016
expiration_threshold = 600

arrival_rate = 0.9
max_time = 300

# number of simulations samples produced
num_sim = 1000

running_function_count = 3
idle_function_count = 5

cold_service_process = ExpSimProcess(rate=cold_service_rate)
warm_service_process = ExpSimProcess(rate=warm_service_rate)

def generate_trace():
    idle_functions = []
    for _ in range(idle_function_count):
        f = FunctionInstance(0,
                                cold_service_process,
                                warm_service_process,
                                expiration_threshold
                                )

        f.state = 'IDLE'
        f.is_cold = False
        # when will it be destroyed if no requests
        f.next_termination = 300
        # so that they would be less likely to be chosen by scheduler
        f.creation_time = 0.01
        idle_functions.append(f)

    running_functions = []
    for _ in range(running_function_count):
        f = FunctionInstance(0,
                                cold_service_process,
                                warm_service_process,
                                expiration_threshold
                                )

        f.state = 'IDLE'
        f.is_cold = False
        # transition it into running mode
        f.arrival_transition(0)

        running_functions.append(f)

    sim = ServerlessTemporalSimulator(running_functions, idle_functions, arrival_rate=arrival_rate, warm_service_rate=warm_service_rate, cold_service_rate=cold_service_rate,
                                        expiration_threshold=expiration_threshold, max_time=max_time)
    sim.generate_trace(debug_print=False, progress=False)
    return sim.get_cold_start_prob()

In [2]:
traces = [generate_trace() for _ in tqdm(range(num_sim))]

100%|██████████| 1000/1000 [00:14<00:00, 70.81it/s]


In [3]:
import numpy as np

p_cold = np.mean(traces)

print(f"The probability of cold start request in the next {max_time}s is: {p_cold:.8f}")

The probability of cold start request in the next 300s is: 0.00035660


## Distribute Workload using ZeroMQ

In this section, we will be using ZeroMQ to distribute workload of generating temporal traces
among distributed workers. Doing so, gives us the ability to get a much higher throughput of
simulations.

In [1]:
import sys
import time
import random
from threading import Thread

import zmq

port = "5556"


context = zmq.Context()
socket = context.socket(zmq.DEALER)
socket.setsockopt(zmq.IDENTITY, b'master')
socket.bind("tcp://127.0.0.1:%s" % port)

poller = zmq.Poller()
poller.register(socket, zmq.POLLIN)

In [2]:
# send the tasks
for _ in range(10):
    request = b"HI"
    ident = b'A'
    # socket.send_multipart([ident, request])
    socket.send(request)

In [3]:
# receive the results
while True:
    socks = dict(poller.poll(timeout=5000))
    if socks == {}:
        break

    if socket in socks and socks[socket] == zmq.POLLIN:
        print("Message from socket: %s" % socket.recv())
        # worker, message = socket.recv_multipart()
        # print(f"Message from {worker.decode()}: {message}")

Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
Message from socket: b'World from 5556'
